In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
import requests
import bs4

In [5]:
alt.renderers.set_embed_options(
    padding={"left": -10, "right": 0, "bottom": 0, "top": 0}
)

RendererRegistry.enable('default')

In [6]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [7]:
from df2gspread import df2gspread as d2g

In [8]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [9]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [10]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-credentials.json", scope
)
gc = gspread.authorize(credentials)

In [11]:
spreadsheet_key = "1x0SgJ83gfsVSvyb7p19BH7Mg5b14EfAFEYRLBj2DLO4"
book = gc.open_by_key(spreadsheet_key)

In [12]:
## Example getting
# Opening the worksheet by using Worksheet ID
# workbook = gc.open_by_key(spreadsheet_key)
# Selecting which sheet to pulling the data
# sheet = workbook.worksheet('Sheet1')
# Pulling the data and transform it to the data frame
# values = sheet.get_all_values()
# pacifictemps = pd.DataFrame(values[1:], columns = values[0])

### current abortion bans

In [13]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

In [14]:
request = requests.get(
    "https://states.guttmacher.org/policies/guttmacher.json", headers=headers
)
data = request.json()

In [15]:
guttmacher_src = pd.DataFrame(data)

In [16]:
guttmacher_src.head()

,state,state_slug,environment_type,policies_currently_in_effect,wora_in_state_15_49,wora_in_state_15_49_breakdown,residents_in_state_born_outside_us,income_level_below_fpl,insurance_coverage_breakdown,wora_age_breakdown,abortions_obtained_in_2017,abortions_per_1000_women_aged_15_44_in_2017,clinics_provided_abortion_in_2017,counties_without_abortion_provider_in_2017,residents_in_counties_without_an_abortion_provider_in_2017,aowdd_15_49_6_weeks,aowdd_15_49_12_weeks,aowdd_15_49_14_weeks,aowdd_15_49_after_14_weeks,aowdd_15_49_20_weeks,aowdd_15_49_22_weeks,aowdd_15_49_24_weeks,aowdd_15_49_after_24_weeks
0,Alabama,alabama,Most Restrictive,Abortion is completely banned with very limite...,1.1 million,Hispanic: 5% | White: 60% | Black: 29% | Asian...,4,35,Uninsured: 14% | Medicaid: 16% | Private: 66% ...,15–17: 8% | 18–19: 6% | 20–24: 14% | 25–29:...,"5,700",6.0,5,93,59,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,Average one-way driving distance for women age...,Average one-way driving distance for women age...
1,Alaska,alaska,Protective,Only physicians can provide abortions and not ...,"161,000",Hispanic: 8% | White: 52% | Black: 3% | Asian:...,10,21,Uninsured: 12% | Medicaid: 18% | Private: 58% ...,15–17: 8% | 18–19: 4% | 20–24: 10% | 25–29:...,"1,240",8.6,4,86,32,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,Abortions after 14 weeks are not available at ...,,,,
2,Arizona,arizona,Restrictive,Pre-Roe abortion ban is still in place and can...,1.6 million,Hispanic: 38% | White: 45% | Black: 5% | Asian...,15,33,Uninsured: 15% | Medicaid: 21% | Private: 62% ...,15–17: 8% | 18–19: 6% | 20–24: 15% | 25–29:...,"13,320",9.3,8,80,18,,,,,Average one-way driving distance for women age...,,Average one-way driving distance for women age...,Average one-way driving distance for women age...
3,Arkansas,arkansas,Most Restrictive,Abortion is completely banned with very limite...,"674,000",Hispanic: 9% | White: 65% | Black: 17% | Asian...,6,41,Uninsured: 12% | Medicaid: 28% | Private: 57% ...,15–17: 9% | 18–19: 6% | 20–24: 14% | 25–29:...,"3,250",5.6,3,97,77,,,,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,,Average one-way driving distance for women age...
4,California,california,Protective,"Abortion is banned at fetal viability, general...",9.2 million,Hispanic: 44% | White: 30% | Black: 5% | Asian...,28,28,Uninsured: 9% | Medicaid: 25% | Private: 64% |...,15–17: 8% | 18–19: 5% | 20–24: 14% | 25–29:...,"154,060",19.2,161,40,3,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,,Average one-way driving distance for women age...,Average one-way driving distance for women age...,,Average one-way driving distance for women age...


In [17]:
wks_name = "overview"
d2g.upload(
    guttmacher_src, spreadsheet_key, wks_name, credentials=credentials, row_names=False
)

<Worksheet 'overview' id:0>

### parental consent

In [18]:
url = "https://www.guttmacher.org/state-policy/explore/parental-involvement-minors-abortions"

In [19]:
page = requests.get(url)

In [20]:
soup = bs4.BeautifulSoup(page.text, "html.parser")

In [27]:
table = soup.find("table")

In [29]:
rows = table.find_all("tr")

In [30]:
df = pd.read_html(str(table), header=2)[0]

In [31]:
df.columns = [
    "state",
    "consent_only",
    "notif_and_consent",
    "notification_only",
    "other_relatives",
    "id",
    "parental_proof",
    "minor_id",
]

In [32]:
df

,state,consent_only,notif_and_consent,notification_only,other_relatives,id,parental_proof,minor_id
0,Alabama,X,NaN,NaN,NaN,X,X,NaN
1,Alaska,NaN,NaN,▼,NaN,NaN,NaN,NaN
2,Arizona,X,NaN,NaN,NaN,X*,NaN,NaN
3,Arkansas,X,NaN,NaN,NaN,X*,NaN,NaN
4,California,▼,NaN,NaN,NaN,NaN,NaN,NaN
5,Colorado,NaN,NaN,X,NaN,NaN,NaN,NaN
6,Delaware‡​,NaN,NaN,X†​,X,NaN,NaN,NaN
7,Florida,NaN,X,NaN,NaN,X,NaN,NaN
8,Georgia,NaN,NaN,X,NaN,X,NaN,NaN
9,Idaho,X,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
wks_name = "parental_consent"
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=False)

<Worksheet 'parental_consent' id:1683475496>

### bans and exceptions

In [34]:
exceptions_url = (
    "https://www.guttmacher.org/state-policy/explore/state-policies-later-abortions#"
)

In [35]:
exceptions_page = requests.get(exceptions_url)

In [36]:
excp_soup = bs4.BeautifulSoup(exceptions_page.text, "html.parser")

In [41]:
excp_table = excp_soup.find("table")

In [48]:
exceptions_df = pd.read_html(str(excp_table), header=2)[0]

In [49]:
exceptions_df.columns = [
    "statutory_limit",
    "state",
    "life",
    "general_helath",
    "physical_health",
    "other",
]

In [50]:
exceptions_df["statutory_limit"] = exceptions_df["statutory_limit"].fillna(
    method="ffill"
)

In [51]:
ban_exceptions = exceptions_df[exceptions_df["statutory_limit"] != "TOTAL IN EFFECT"]

In [52]:
ban_exceptions.columns = ban_exceptions.columns.str.lower().str.replace(" ", "_")

In [53]:
ban_exceptions

,statutory_limit,state,life,general_helath,physical_health,other
0,Conception,Alabama,X,NaN,X,NaN
1,Conception,Arizona,X,NaN,NaN,NaN
2,Conception,Arkansas,X,NaN,NaN,NaN
3,Conception,Idaho,X,NaN,X,"Rape, incest"
4,Conception,Kentucky,X,NaN,X,NaN
5,Conception,Louisiana,X,NaN,X,Lethal fetal anomaly
6,Conception,Mississippi,X,NaN,NaN,rape
7,Conception,Missouri,X,NaN,X,NaN
8,Conception,Oklahoma,X,NaN,NaN,NaN
9,Conception,South Dakota,X,NaN,NaN,NaN


In [54]:
wks_name = "gestationalage_exceptions"
d2g.upload(
    ban_exceptions, spreadsheet_key, wks_name, credentials=credentials, row_names=False
)

<Worksheet 'gestationalage_exceptions' id:690368369>

### trying to get the data from kff's abortion tracker--you can download from the inspector but get a 403 when i try to use the link

In [ ]:
#pd.read_csv('https://datawrapper.dwcdn.net/Q43DW/37/dataset.csv')